Q1: L'advantage de MeanShift :
    1) il y a moins de calcul;
    2) le temps d'éxecution est petit
    3) il peut suivre l'objet rapidement
   L'inconvénient est qu'il ne peut pas suivre l'objet précisément. Si l'objet se deplace très rapidement, le fenetre perd son objet. 

In [1]:
import numpy as np
import cv2

roi_defined = False
 
def define_ROI(event, x, y, flags, param):
	global r,c,w,h,roi_defined
	# if the left mouse button was clicked, 
	# record the starting ROI coordinates 
	if event == cv2.EVENT_LBUTTONDOWN:
		r, c = x, y
		roi_defined = False
	# if the left mouse button was released,
	# record the ROI coordinates and dimensions
	elif event == cv2.EVENT_LBUTTONUP:
		r2, c2 = x, y
		h = abs(r2-r)
		w = abs(c2-c)
		r = min(r,r2)
		c = min(c,c2)  
		roi_defined = True

cap = cv2.VideoCapture('Antoine_Mug.mp4')

# take first frame of the video
ret,frame = cap.read()
# load the image, clone it, and setup the mouse callback function
clone = frame.copy()
cv2.namedWindow("First image")
cv2.setMouseCallback("First image", define_ROI)
 
# keep looping until the 'q' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("First image", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the ROI is defined, draw it!
	if (roi_defined):
		# draw a green rectangle around the region of interest
		cv2.rectangle(frame, (r,c), (r+h,c+w), (0, 255, 0), 2)
	# else reset the image...
	else:
		frame = clone.copy()
	# if the 'q' key is pressed, break from the loop
	if key == ord("q"):
		break
 
track_window = (r,c,h,w)
# set up the ROI for tracking
roi = frame[c:c+w, r:r+h]
# conversion to Hue-Saturation-Value space
# 0 < H < 180 ; 0 < S < 255 ; 0 < V < 255
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# computation mask of the histogram:
# Pixels with S<30, V<20 or V>235 are ignored 
mask = cv2.inRange(hsv_roi, np.array((0.,30.,20.)), np.array((180.,255.,235.)))
# Marginal histogram of the Hue component
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Histogram values are normalised to [0,255]
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria: either 10 iterations,
# or move by less than 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

cpt = 1
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	# Backproject the model histogram roi_hist onto the 
	# current image hsv, i.e. dst(x,y) = roi_hist(hsv(0,x,y))
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to dst to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw a blue rectangle on the current image
        r,c,h,w = track_window
        frame_tracked = cv2.rectangle(frame, (r,c), (r+h,c+w), (255,0,0) ,2)
        cv2.imshow('Sequence',frame_tracked)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('Frame_%04d.png'%cpt,frame_tracked)
        cpt += 1
    else:
        break

cv2.destroyAllWindows()
cap.release()

Q2: Je trouve que dans l'image de rétroprojection les pixels de la porte ont les valeur plus élevés que les pixels de tasse. Donc, il perd son objet quand il suit la tasse. Pour amélioer, je change le canal de HSV de canal H(hue) à S(saturation).
    roi_hist = cv2.calcHist([hsv_roi],[1],mask,[180],[0,180])

In [2]:
import numpy as np
import cv2

roi_defined = False
 
def define_ROI(event, x, y, flags, param):
	global r,c,w,h,roi_defined
	# if the left mouse button was clicked, 
	# record the starting ROI coordinates 
	if event == cv2.EVENT_LBUTTONDOWN:
		r, c = x, y
		roi_defined = False
	# if the left mouse button was released,
	# record the ROI coordinates and dimensions
	elif event == cv2.EVENT_LBUTTONUP:
		r2, c2 = x, y
		h = abs(r2-r)
		w = abs(c2-c)
		r = min(r,r2)
		c = min(c,c2)  
		roi_defined = True

cap = cv2.VideoCapture('Antoine_Mug.mp4')

# take first frame of the video
ret,frame = cap.read()
# load the image, clone it, and setup the mouse callback function
clone = frame.copy()
cv2.namedWindow("First image")
cv2.setMouseCallback("First image", define_ROI)
 
# keep looping until the 'q' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("First image", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the ROI is defined, draw it!
	if (roi_defined):
		# draw a green rectangle around the region of interest
		cv2.rectangle(frame, (r,c), (r+h,c+w), (0, 255, 0), 2)
	# else reset the image...
	else:
		frame = clone.copy()
	# if the 'q' key is pressed, break from the loop
	if key == ord("q"):
		break
 
track_window = (r,c,h,w)
# set up the ROI for tracking
roi = frame[c:c+w, r:r+h]
# conversion to Hue-Saturation-Value space
# 0 < H < 180 ; 0 < S < 255 ; 0 < V < 255
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# computation mask of the histogram:
# Pixels with S<30, V<20 or V>235 are ignored 
mask = cv2.inRange(hsv_roi, np.array((0.,30.,20.)), np.array((180.,255.,235.)))
# Marginal histogram of the Hue component
#roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])

#Q2
roi_hist = cv2.calcHist([hsv_roi],[1],mask,[180],[0,180])

# Histogram values are normalised to [0,255]
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria: either 10 iterations,
# or move by less than 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

cpt = 1
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	# Backproject the model histogram roi_hist onto the 
	# current image hsv, i.e. dst(x,y) = roi_hist(hsv(0,x,y))
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #Q2 display for result 
        cv2.imshow('Dis ',dst)
        
        # apply meanshift to dst to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw a blue rectangle on the current image
        r,c,h,w = track_window
        frame_tracked = cv2.rectangle(frame, (r,c), (r+h,c+w), (255,0,0) ,2)
        cv2.imshow('Sequence',frame_tracked)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('Frame_%04d.png'%cpt,frame_tracked)
        cpt += 1
    else:
        break

cv2.destroyAllWindows()
cap.release()

Q3: Pour calculer l'orientation du gradient, il faut calculer la direction horizontale et la direction verticale avec deux coeurs. Ensuite, j'utilise la fonction cv2.magnitude() pour calculer le norm du gradient. Puis, je peinte les pixels qui sont moins que 50 avec la couleur rouge. Donc il y a cinq image affichantes : 1) "first image", 2) "sequence image", 3)"orientation du gradient", 4)"norm du gradient", 5)"orientation" (image avec mask).

In [5]:
import numpy as np
import cv2

roi_defined = False
 
def define_ROI(event, x, y, flags, param):
	global r,c,w,h,roi_defined
	# if the left mouse button was clicked, 
	# record the starting ROI coordinates 
	if event == cv2.EVENT_LBUTTONDOWN:
		r, c = x, y
		roi_defined = False
	# if the left mouse button was released,
	# record the ROI coordinates and dimensions
	elif event == cv2.EVENT_LBUTTONUP:
		r2, c2 = x, y
		h = abs(r2-r)
		w = abs(c2-c)
		r = min(r,r2)
		c = min(c,c2)  
		roi_defined = True

cap = cv2.VideoCapture('Antoine_Mug.mp4')

# take first frame of the video
ret,frame = cap.read()
# load the image, clone it, and setup the mouse callback function
clone = frame.copy()
cv2.namedWindow("First image")
cv2.setMouseCallback("First image", define_ROI)
 
# keep looping until the 'q' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("First image", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the ROI is defined, draw it!
	if (roi_defined):
		# draw a green rectangle around the region of interest
		cv2.rectangle(frame, (r,c), (r+h,c+w), (0, 255, 0), 2)
	# else reset the image...
	else:
		frame = clone.copy()
	# if the 'q' key is pressed, break from the loop
	if key == ord("q"):
		break
 
track_window = (r,c,h,w)
# set up the ROI for tracking
roi = frame[c:c+w, r:r+h]
# conversion to Hue-Saturation-Value space
# 0 < H < 180 ; 0 < S < 255 ; 0 < V < 255
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# computation mask of the histogram:
# Pixels with S<30, V<20 or V>235 are ignored 
mask = cv2.inRange(hsv_roi, np.array((0.,30.,20.)), np.array((180.,255.,235.)))
# Marginal histogram of the Hue component
roi_hist = cv2.calcHist([hsv_roi],[1],mask,[180],[0,180])
# Histogram values are normalised to [0,255]
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria: either 10 iterations,
# or move by less than 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

cpt = 1
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	# Backproject the model histogram roi_hist onto the 
	# current image hsv, i.e. dst(x,y) = roi_hist(hsv(0,x,y))
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to dst to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw a blue rectangle on the current image
        r,c,h,w = track_window
        frame_tracked = cv2.rectangle(frame, (r,c), (r+h,c+w), (255,0,0) ,2)
        cv2.imshow('Sequence',frame_tracked)

        ##Q3
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        
        #orientation x
        kernel = np.array([[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]])
        Ix = cv2.filter2D(frame_gray,-1,kernel)
        Ix = np.array(Ix, np.float32)
        #oritentaion y
        kernel = np.array([[-1, -1, -1],[0, 0, 0],[1, 1, 1]])
        Iy = cv2.filter2D(frame_gray,-1,kernel)
        Iy = np.array(Iy, np.float32)
        #angle calculation
        angle = cv2.phase(Ix,Iy,angleInDegrees=True)
        #norm of gradient
        norm = cv2.magnitude(Ix, Iy)
        #add mask
        thresh = 50
        _, mask = cv2.threshold(norm, thresh, 255, cv2.THRESH_BINARY)
        orient = np.zeros((angle.shape[0], angle.shape[1], 3),dtype=np.float32)
        orient[(mask == 0)] = np.array([0, 0, 1])#fill with red color
        #display of result
        cv2.imshow('Orientation du gradient',angle)
        cv2.imshow('Norm du gradient',norm)
        cv2.imshow('Orientation',orient)
        
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('Frame_%04d.png'%cpt,frame_tracked)
        cpt += 1
    else:
        break

cv2.destroyAllWindows()
cap.release()

Q4: Pour achievez la transformation hough,je définis quatre fonctions:
    1)orientation_gradient(image): calculez l'orientation du gradient, returnez l'angle (dégrée)
    2)RTable(image,ref): il faut un image grey et un point de réference. D'about il calcule l'orientation du gradient avec la fonction orientation_gradient(image). Ensuite, il construit une RTable pour ceux qui ont la même angle avec un vecteur entre ce point et le point de réference.
    3)detection(image,rTable): il faut utiliser le résultat de fonction RTable(image,ref). Il trouve la similarité entre cet image et la RTable de l'autre image par l'orientation du gradient.Pour tous les points qui ont la même orientation, augmentez la graphique de vote.
    4)Hmax(hough): selectez la position plus possible par choisir arg max H(x), retournez la valeur maximale et la position(x,y)

In [ ]:
import numpy as np
import cv2
from collections import defaultdict
    
def orientation_gradient(image):
    Image = image
    #orientation x
    kernel = np.array([[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]])
    Ix = cv2.filter2D(image,-1,kernel)
    Ix = np.array(Ix, np.float32)
    #oritentaion y
    kernel = np.array([[-1, -1, -1],[0, 0, 0],[1, 1, 1]])
    Iy = cv2.filter2D(image,-1,kernel)
    Iy = np.array(Iy, np.float32)
    #angle calculation
    angle = cv2.phase(Ix,Iy,angleInDegrees=True)
    angle = angle*180/np.pi #change to degree
    return angle

def RTable(image,ref):#need gray image and reference point(x,y)
    Image = image
    edge = cv2.Canny(Image,10,1000)#find the line in the image
    angle = orientation_gradient(image)
    rTable = defaultdict(list)
    for (i,j),value in np.ndenumerate(edge):
        rTable[angle[i,j]].append((ref[0]-i,ref[1]-j))#store the vector in each angle
    return rTable

def detection(image,rTable):#construct the vote map H(x)
    Image = image
    edge = cv2.Canny(image,10,1000)#find the line in the image
    angle = orientation_gradient(image)

    hough = np.zeros(image.shape)
    for (i,j),value in np.ndenumerate(edge):
        for r in rTable[angle[i,j]]:
            x = r[0] + i
            y = r[1] + j
            if x < hough.shape[0] and y < hough.shape[1]:
                hough[i,j] += 1 #suppose weight is 1
    return hough

def Hmax(hough):#find the arg max H(x)
    #x = np.argmax(hough, axis=1)
    #y =  np.max(hough, axis=1)
    #print(x,y)
    #return x,y
    indices = hough.ravel().argsort()[-1:]
    indices = (np.unravel_index(i, hough.shape) for i in indices)
    #print([(hough[i], i) for i in indices])
    return [(hough[i], i) for i in indices]
                
## Test Code        
  
roi_defined = False
 
def define_ROI(event, x, y, flags, param):
	global r,c,w,h,roi_defined
	# if the left mouse button was clicked, 
	# record the starting ROI coordinates 
	if event == cv2.EVENT_LBUTTONDOWN:
		r, c = x, y
		roi_defined = False
	# if the left mouse button was released,
	# record the ROI coordinates and dimensions
	elif event == cv2.EVENT_LBUTTONUP:
		r2, c2 = x, y
		h = abs(r2-r)
		w = abs(c2-c)
		r = min(r,r2)
		c = min(c,c2)  
		roi_defined = True

cap = cv2.VideoCapture('Antoine_Mug.mp4')

# take first frame of the video
ret,frame = cap.read()
# load the image, clone it, and setup the mouse callback function
clone = frame.copy()
cv2.namedWindow("First image")
cv2.setMouseCallback("First image", define_ROI)
 
# keep looping until the 'q' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("First image", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the ROI is defined, draw it!
	if (roi_defined):
		# draw a green rectangle around the region of interest
		cv2.rectangle(frame, (r,c), (r+h,c+w), (0, 255, 0), 2)
	# else reset the image...
	else:
		frame = clone.copy()
	# if the 'q' key is pressed, break from the loop
	if key == ord("q"):
		break
 
track_window = (r,c,h,w)
# set up the ROI for tracking
roi = frame[c:c+w, r:r+h]
# conversion to Hue-Saturation-Value space
# 0 < H < 180 ; 0 < S < 255 ; 0 < V < 255
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# computation mask of the histogram:
# Pixels with S<30, V<20 or V>235 are ignored 
mask = cv2.inRange(hsv_roi, np.array((0.,30.,20.)), np.array((180.,255.,235.)))
# Marginal histogram of the Hue component
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Histogram values are normalised to [0,255]
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria: either 10 iterations,
# or move by less than 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

##Q4
referenceImage = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
sobelX = cv2.Sobel(referenceImage,cv2.CV_64F,1,0)#gradient X
sobelY = cv2.Sobel(referenceImage,cv2.CV_64F,0,1)#gradient Y

sobelX = np.uint8(np.absolute(sobelX))
sobelY = np.uint8(np.absolute(sobelY))

RefSobelCombined = cv2.bitwise_or(sobelX,sobelY)
referencePoint = [RefSobelCombined.shape[0]/2, RefSobelCombined.shape[1]/2]#choose center of reference image as the reference point
rTable = RTable(RefSobelCombined,referencePoint)

cpt = 1
while(1):
    ret ,frame = cap.read()
    if ret == True:
        ##Q4
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sobelX = cv2.Sobel(frame_gray,cv2.CV_64F,1,0)#gradient X
        sobelY = cv2.Sobel(frame_gray,cv2.CV_64F,0,1)#gradient Y

        sobelX = np.uint8(np.absolute(sobelX))
        sobelY = np.uint8(np.absolute(sobelY))

        sobelCombined = cv2.bitwise_or(sobelX,sobelY)
        hough = detection(sobelCombined,rTable)
        m =  Hmax(hough)
        y = [pt[1][0] for pt in m][0]
        x = [pt[1][1] for pt in m][0]
        cv2.imshow('Hough',hough)

        frame_tracked = cv2.rectangle(frame, (int(x-0.5*h),int(y-0.5*w)), (int(x+0.5*h),int(y+0.5*w)), (255,0,0) ,2)
        #cv2.imshow('Sequence',frame_tracked)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('Frame_%04d.png'%cpt,frame_tracked)
        cpt += 1
    else:
        break

cv2.destroyAllWindows()
cap.release()


Q5

In [ ]:
import numpy as np
import cv2
from collections import defaultdict
    
def orientation_gradient(image):
    Image = image
    #orientation x
    kernel = np.array([[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]])
    Ix = cv2.filter2D(image,-1,kernel)
    Ix = np.array(Ix, np.float32)
    #oritentaion y
    kernel = np.array([[-1, -1, -1],[0, 0, 0],[1, 1, 1]])
    Iy = cv2.filter2D(image,-1,kernel)
    Iy = np.array(Iy, np.float32)
    #angle calculation
    angle = cv2.phase(Ix,Iy,angleInDegrees=True)
    angle = angle*180/np.pi #change to degree
    return angle

def RTable(image,ref):#need gray image and reference point(x,y)
    Image = image
    edge = cv2.Canny(Image,10,1000)#find the line in the image
    angle = orientation_gradient(image)
    rTable = defaultdict(list)
    for (i,j),value in np.ndenumerate(edge):
        rTable[angle[i,j]].append((ref[0]-i,ref[1]-j))#store the vector in each angle
    return rTable

def detection(image,rTable):#construct the vote map H(x)
    Image = image
    edge = cv2.Canny(image,10,1000)#find the line in the image
    angle = orientation_gradient(image)

    hough = np.zeros(image.shape)
    for (i,j),value in np.ndenumerate(edge):
        for r in rTable[angle[i,j]]:
            x = r[0] + i
            y = r[1] + j
            if x < hough.shape[0] and y < hough.shape[1]:
                hough[i,j] += 1 #suppose weight is 1
    return hough

def Hmax(hough):#find the arg max H(x)
    #x = np.argmax(hough, axis=1)
    #y =  np.max(hough, axis=1)
    #print(x,y)
    #return x,y
    indices = hough.ravel().argsort()[-1:]
    indices = (np.unravel_index(i, hough.shape) for i in indices)
    #print([(hough[i], i) for i in indices])
    return [(hough[i], i) for i in indices]
                
## Test Code        
  
roi_defined = False
 
def define_ROI(event, x, y, flags, param):
	global r,c,w,h,roi_defined
	# if the left mouse button was clicked, 
	# record the starting ROI coordinates 
	if event == cv2.EVENT_LBUTTONDOWN:
		r, c = x, y
		roi_defined = False
	# if the left mouse button was released,
	# record the ROI coordinates and dimensions
	elif event == cv2.EVENT_LBUTTONUP:
		r2, c2 = x, y
		h = abs(r2-r)
		w = abs(c2-c)
		r = min(r,r2)
		c = min(c,c2)  
		roi_defined = True

cap = cv2.VideoCapture('Antoine_Mug.mp4')

# take first frame of the video
ret,frame = cap.read()
# load the image, clone it, and setup the mouse callback function
clone = frame.copy()
cv2.namedWindow("First image")
cv2.setMouseCallback("First image", define_ROI)
 
# keep looping until the 'q' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("First image", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the ROI is defined, draw it!
	if (roi_defined):
		# draw a green rectangle around the region of interest
		cv2.rectangle(frame, (r,c), (r+h,c+w), (0, 255, 0), 2)
	# else reset the image...
	else:
		frame = clone.copy()
	# if the 'q' key is pressed, break from the loop
	if key == ord("q"):
		break
 
track_window = (r,c,h,w)
# set up the ROI for tracking
roi = frame[c:c+w, r:r+h]
# conversion to Hue-Saturation-Value space
# 0 < H < 180 ; 0 < S < 255 ; 0 < V < 255
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# computation mask of the histogram:
# Pixels with S<30, V<20 or V>235 are ignored 
mask = cv2.inRange(hsv_roi, np.array((0.,30.,20.)), np.array((180.,255.,235.)))
# Marginal histogram of the Hue component
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Histogram values are normalised to [0,255]
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria: either 10 iterations,
# or move by less than 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

##Q4
referenceImage = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
referencePoint = [referenceImage.shape[0]/2, referenceImage.shape[1]/2]#choose center of reference image as the reference point
rTable = RTable(referenceImage,referencePoint)

cpt = 1
while(1):
    ret ,frame = cap.read()
    if ret == True:
        ##Q4
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hough = detection(frame_gray,rTable)

        # apply meanshift to dst to get the new location
        ret, track_window = cv2.meanShift(hough, track_window, term_crit)

        # Draw a blue rectangle on the current image
        r,c,h,w = track_window
        frame_tracked = cv2.rectangle(frame, (r,c), (r+h,c+w), (255,0,0) ,2)
        cv2.imshow('Sequence',frame_tracked)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('Frame_%04d.png'%cpt,frame_tracked)
        cpt += 1
    else:
        break

cv2.destroyAllWindows()
cap.release()